In [2]:
import numpy as np
import pandas as pd
import more_itertools as mit
import os

In [4]:
csv_direc = 'C:\\Users\\Ben\\OneDrive - Duke University\\bilbo_lab\\il34_project\\il34_behavior\\sociability\\IL34_SOC1\\analysis\\csv_output\\'

In [223]:
df_og = pd.read_csv('C:\\Users\\Ben\\OneDrive - Duke University\\bilbo_lab\\il34_project\\il34_behavior\\sociability\\IL34_SOC1\\analysis\\csv_output\\IL34_SOC1_1DLC_resnet_50_social_behavior_allMay27shuffle1_250000.csv', header = [1, 2])

In [6]:
### first step is to find a way to extract the groups of index values where the network was uncertain about the coordinates (likelihood < .95, this can be tweaked)

def extract_uncertain_groups(df, bodypart = 'nose', uncertainty = 0.95):
    iterable = df[df[bodypart]['likelihood'] < uncertainty].index.values
    return [list(group) for group in mit.consecutive_groups(iterable)]

uncertain_ilocs = extract_uncertain_groups(df, uncertainty = .95)

### we now have a list of index location groups where the network was uncertain about the nose

In [7]:
### building a function to increment the averages between the two values

def increment_average(two_vals, inc_len):
    
    increment = (two_vals[1] - two_vals[0]) / (inc_len + 2)
    vals = []
    temp = two_vals[0]
    for i in range(inc_len):
        temp = temp + increment
        vals = np.append(vals, temp)
        
    return vals

In [170]:
### next step is to go through these groups, one by one, and extract out a mini dataframe that includes the uncertain coordinates
### as well as the coordinates 1 frame BEFORE it was uncertain, and 1 frame AFTER it becomes certain again

for group in uncertain_ilocs:
    ### indexing the mini dataframe, and assign it to a temporary variable
    #df_subset = df.loc[int(group[0] - 1): int(group[-1] + 1)]
    
    ### we also want to get the length of the group, so that we can smooth the average by this value
    #print(len(group))
    
    ### next, we want to average the bodypart 'x' coordinates, and bodypart 'y' coordinates in between
    try:
        #print(increment_average([df_subset['nose']['x'][group[0] - 1], df_subset['nose']['x'][group[-1] + 1]], len(group)))
        #print(df['nose']['x'].loc[int(group[0]): int(group[-1])].values)
        ### replacing the values with the new, adjusted values in the original dataframe
        df.replace(df['nose']['x'][group].values, increment_average([df['nose']['x'][group[0] - 1], df['nose']['x'][group[-1] + 1]], len(group)), inplace = True)
       
    except:
        pass

In [8]:
### alright, now let's try to put it all together, including all bodyparts, and both x and y

df = pd.read_csv(csv_direc + 'IL34_SOC1_1DLC_resnet_50_social_behavior_allMay27shuffle1_250000.csv',  header = [1, 2])

### need to define a list of all the bodyparts and the two coordinates
bodyparts = ['nose', 'right ear', 'left ear', 'tail']
coordinates = ['x', 'y']

### loop through the bodyparts
for bodypart in bodyparts:
    ### step 1, extract uncertainties 
    uncertain_ilocs = extract_uncertain_groups(df, uncertainty = .95, bodypart = bodypart)
    ### step 2, loop through x and y coordinates, assign incrementally averaged values
    for coord in coordinates:
        print('Smoothing out: ', bodypart + ' ' + coord)
        for group in uncertain_ilocs:
            try:
                df.replace(df[bodypart][coord][group].values, increment_average([df[bodypart][coord][group[0] - 1], df[bodypart][coord][group[-1] + 1]], len(group)), inplace = True)
            except:
                pass

Smoothing out:  nose x
Smoothing out:  nose y
Smoothing out:  right ear x
Smoothing out:  right ear y
Smoothing out:  left ear x
Smoothing out:  left ear y
Smoothing out:  tail x
Smoothing out:  tail y


In [10]:
df.set_index('bodyparts')

ValueError: Index data must be 1-dimensional

In [157]:
### replacing the values with the new, adjusted values in the original dataframe

df.replace(df['nose']['x'][uncertain_ilocs[2]].values, increment_average([df['nose']['x'][uncertain_ilocs[2][0] - 1], df['nose']['x'][uncertain_ilocs[2][-1] + 1]], len(uncertain_ilocs[2])), inplace = True)

In [8]:
def process_csv_backwards(df):
    nosex = df['nose']['x'].values
    nosey = df['nose']['y'].values

    leftx = df['left ear']['x'].values
    lefty = df['left ear']['y'].values

    rightx = df['right ear']['x'].values
    righty = df['right ear']['y'].values

    tailx = df['tail']['x'].values
    taily = df['tail']['y'].values

    for i in range(len(df) - 1):
        i = i+1
        if df['nose']['likelihood'].loc[i] < 0.98:
            nosex[i] = nosex[i-1]
            nosey[i] = nosey[i-1]
            tailx[i] = tailx[i-1]
            taily[i] = taily[i-1]

        if df['right ear']['likelihood'].loc[i] < 0.98:
            rightx[i] = rightx[i-1]
            righty[i] = righty[i-1]

        if df['left ear']['likelihood'].loc[i] < 0.98:
            leftx[i] = leftx[i-1]
            lefty[i] = lefty[i-1]
            
    df_processed = pd.DataFrame([nosex, nosey, rightx, righty, leftx, lefty, tailx, taily]).T
    colnames = df.T.loc[[('nose', 'x'), ('nose', 'y'), ('right ear', 'x'), ('right ear', 'y'), ('left ear', 'x'), ('left ear', 'y'), ('tail', 'x'), ('tail', 'y')]].T.columns
    
    df_processed.columns = colnames
    
    return df_processed


In [22]:
df = pd.read_csv('testing.csv', header = [0, 1]).dropna()

In [23]:
df

Unnamed: 0_level_0        nose                        right ear  \
      Unnamed: 0_level_1           x           y likelihood         x   
1                      0   -3.151770   -1.986736   0.000011 -2.298127   
2                      1   -3.252387   -2.070119   0.000011 -2.257833   
3                      2   -3.505652   -2.026593   0.000007 -2.523723   
4                      3   -3.235620   -2.088121   0.000011 -2.150948   
5                      4  337.047211  327.373657   0.000040 -2.319052   
...                  ...         ...         ...        ...       ...   
22740              22739   -2.946157   -2.352730   0.000019 -2.634086   
22741              22740   -2.966658   -2.425992   0.000020 -2.718083   
22742              22741   -3.195249   -2.297779   0.000022 -2.519485   
22743              22742   -3.182985   -2.432640   0.000020 -2.361449   
22744              22743   -3.448080   -2.268462   0.000026 -2.361680   

                            left ear                           tail            \
              y likelihood         x         y likelihood         x         y   
1     -5.833694   0.000003 -1.297530 -1.772893   0.000005 -0.495781 -0.007596   
2     -5.863381   0.000003 -1.245870 -1.696291   0.000005 -0.476863  0.197253   
3     -5.966809   0.000002 -1.457158 -1.694777   0.000004 -0.519907 -0.331736   
4     -5.814690   0.000003 -1.317257 -1.735174   0.000005 -0.428813  0.094779   
5     -5.627332   0.000003 -1.183522 -1.890190   0.000005 -0.171284 -0.112352   
...         ...        ...       ...       ...        ...       ...       ...   
22740 -7.051517   0.000008 -0.173042 -1.421008   0.000003 -0.150004  1.593057   
22741 -6.904866   0.000008  0.216149 -1.441775   0.000003 -0.172407  1.420202   
22742 -6.967012   0.000009 -0.576201 -1.311594   0.000004 -0.311760  1.496567   
22743 -6.936853   0.000008 -0.596949 -1.442449   0.000004 -0.314631  1.570807   
22744 -7.404264   0.000009 -0.570646 -1.331161   0.000003 -0.429017  1.410066   

                  
      likelihood  
1       0.000002  
2       0.000002  
3       0.000001  
4       0.000002  
5       0.000002  
...          ...  
22740   0.000007  
22741   0.000006  
22742   0.000007  
22743   0.000008  
22744   0.000011  

[22744 rows x 13 columns]